# Tuneo de hiperparametros

## Definicion de funciones auxiliares generales

In [1]:
#Si se corre en Colab
#!pip install category_encoders

In [2]:
#FUENTE: - https://stackoverflow.com/questions/43533610/how-to-use-hyperopt-for-hyperparameter-optimization-of-keras-deep-learning-netwo
#        - https://github.com/keras-team/keras/issues/1591

import pickle
import pandas as pd
import numpy as np
from math import isclose
import Utilidades as ut
import Modelos as md
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt import space_eval
from sklearn.metrics import log_loss
import keras

In [3]:
# El Fold es de tamaño fijo, dividiendo por trimestre.

def get_quarter(month_series):
    quarter_series = ((month_series - 1)//3 + 1)
    return quarter_series

## ATENCION: Aca se esta asumiendo que se hace un split train-validation a partir del 2018.
## Se podria modificar esta funcion como para que chequee que un trimestre tenga al menos X cantidad de filas.
## Ademas esta funcion no tiene en cuenta la cantidad de datos con la que se entrena, prioriza dividir por trimestre.

def fold_split_quarter(df_train):
    folds = list()
    years = [2016, 2017]
    last_year = years[-1]
    
    actual_year = df_train.Opportunity_Created_Date.dt.year
    actual_quarter = get_quarter(df_train.Opportunity_Created_Date.dt.month)
    
    ##Esto de abajo se tiene que hacer si no se desea utilizar 'Opportunity_Created_Date'.
    
    df_train = df_train.drop(columns = ['Opportunity_Created_Date'])
    
    ########################
    # Se puede hacer un segundo for para los trimestres.
    for split_year in years:
        
        to_Q1 = df_train[(actual_year <= split_year) & (actual_quarter <= 1)]
        to_Q2 = df_train[(actual_year <= split_year) & (actual_quarter <= 2)]
        to_Q3 = df_train[(actual_year <= split_year) & (actual_quarter <= 3)]
        to_Q4 = df_train[(actual_year <= split_year) & (actual_quarter <= 4)]
        
        Q1 = df_train[(actual_year == split_year) & (actual_quarter == 1)]
        Q2 = df_train[(actual_year == split_year) & (actual_quarter == 2)]
        Q3 = df_train[(actual_year == split_year) & (actual_quarter == 3)]
        Q4 = df_train[(actual_year == split_year) & (actual_quarter == 4)]
        Q5 = df_train[(actual_year == (split_year + 1)) & (actual_quarter == 1)]
        
        folds.append((to_Q1.copy(), Q2.copy()))
        folds.append((to_Q2.copy(), Q3.copy()))
        folds.append((to_Q3.copy(), Q4.copy()))
        if (split_year != last_year):
            folds.append((to_Q4.copy(), Q5.copy()))
            
    return folds

def prepare_folds(folds):
    
    new_folds = list()
    
    for (df_train, df_test) in folds:
        
        #Separamos labels del set de entrenamiento y test
        df_train_x, df_train_y = ut.split_labels(df_train)
        df_test_x, df_test_y = ut.split_labels(df_test)
        
        #Encoding, conversion de fechas y normalizacion numerica para set de test y train
        df_train_x, df_test_x = ut.conversion_fechas(df_train_x, df_test_x)
        df_train_x, df_test_x = ut.codificar_categoricas(df_train_x, df_train_y, df_test_x, modo='catboost')
        df_train_x, df_test_x = ut.normalizacion_numericas(df_train_x, df_test_x, modo='normalizacion')
        
        #Conversion de los dataframes a vectores.
        x_train = ut.df_a_vector(df_train_x)
        y_train = ut.df_a_vector(df_train_y)
        x_test = ut.df_a_vector(df_test_x)
        y_test = ut.df_a_vector(df_test_y)
        
        new_folds.append((x_train, y_train, x_test, y_test))
        
    return new_folds


def spaces_equal(space_1, space_2):
    equal = True
    rel_tol = 0.01 # 1%
    
    if len(space_1) != len(space_2):
        return False
    
    #Varios
    equal = (equal and (isclose(space_1['learning_rate'], space_2['learning_rate'], rel_tol=rel_tol)))
    equal = (equal and (isclose(space_1['alpha'], space_2['alpha'], rel_tol=rel_tol)))
    equal = (equal and (space_1['optimizer'] == space_2['optimizer']))
    
    #First layer
    equal = (equal and (isclose(space_1['first_layer']['neurons'], space_2['first_layer']['neurons'], rel_tol=rel_tol)))
    equal = (equal and (isclose(space_1['first_layer']['dropout'], space_2['first_layer']['dropout'], rel_tol=rel_tol)))
    equal = (equal and (space_1['first_layer']['activation'] == space_2['first_layer']['activation']))
    
    #Hidden layers
    
    if len(space_1['hidden_layers']) != len(space_2['hidden_layers']):
        return False
    
    if not equal: return False
    
    hidden_layers_2 = space_2['hidden_layers']
    for idx, layer_1 in enumerate(space_1['hidden_layers']):
        if layer_1['config']['is_on'] != hidden_layers_2[idx]['config']['is_on']: return False
        if layer_1['config']['is_on']:
            equal = (equal and (isclose(layer_1['config']['neurons'], hidden_layers_2[idx]['config']['neurons'], rel_tol=rel_tol)))
            equal = (equal and (isclose(layer_1['config']['dropout'], hidden_layers_2[idx]['config']['dropout'], rel_tol=rel_tol)))
            equal = (equal and (layer_1['config']['activation'] == hidden_layers_2[idx]['config']['activation']))
    
    return equal

best_loss = [np.inf]

def test_model(params, fit_model, folds, space, factor_poda=3, last_k_average=5):

    val_loss = list()
    #weights = np.array([0.1, 0.1, 0.1, 0.15, 0.15, 0.2, 0.2])
    weights = np.array([0.05, 0.05, 0.05, 0.10, 0.2, 0.25, 0.3])
    weights = weights * len(weights)
    
    print("-"*50)
    print("Comienzo de iteracion con nuevos parametros")
    print(params)
    #Optimizacion extraida de: https://github.com/hyperopt/hyperopt/issues/370
    if len(trials.trials)>1:
        for x in trials.trials[:-1]:
            space_point_index = dict([(key,value[0]) for key,value in x['misc']['vals'].items() if len(value)>0])
            if spaces_equal(params, space_eval(space,space_point_index)):
                print("Se omite la serie de parametros por repeticion\n")
                loss = x['result']['loss']
                return {'loss': loss, 'status': STATUS_OK}
    ###
    for i, fold in enumerate(folds):
        history = fit_model(fold, params)
        last_val_loss = np.mean(history[-last_k_average:])
        val_loss.append(last_val_loss*weights[i])
        val_loss_mean = np.mean(val_loss)
        print(f"Resultado parcial: val_loss = {last_val_loss}")
        if val_loss_mean >= factor_poda*best_loss[0]:
            print("Se omite serie de parametos por bajo rendimiento\n")
            print(f"\tResultado final (media ponderada): val_loss = {val_loss_mean}\n")
            return {'loss': val_loss_mean, 'status': STATUS_OK}
        
    print(f"\n\tResultado final (media ponderada): val_loss = {val_loss_mean}\n")

    best_loss[0] = min(best_loss[0], val_loss_mean)

    return {'loss': val_loss_mean, 'status': STATUS_OK}


def load_trials(N, model_name):
    #Idea obtenida de: https://github.com/hyperopt/hyperopt/issues/267
    trials = None
    total_iters = N
    try:
        fd = open('Archivos/' + model_name + "_hyperparams.hopt", "rb")
        trials = pickle.load(fd)
        fd.close()
        print("Se encontró un entrenamiento previo. Cargando...")
        total_iters = len(trials.trials) + N
        print("Comienza el tuneo desde {} hasta {} trials".format(len(trials.trials), total_iters))
    except:  
        trials = Trials()
    
    return trials, total_iters

def save_trials(trials, model_name):
    # save the trials object
    print(f"Guardando el entrenamiento en Archivos/'{model_name}_hyperparams.hopt'")
    with open('Archivos/' + model_name + "_hyperparams.hopt", "wb") as f:
        pickle.dump(trials, f)

# Modelos
## Redes Neuronales

In [4]:
#Espacio de busqueda para parametros continuos -> Random o Bayes

neural_network_params = {
            
            'first_layer' : {
                'neurons' : hp.quniform('first_layer_neurons', 40, 100, 5),
                'activation' : hp.choice('first_layer_activation', ['relu', 'tanh', 'swish']),
                'dropout' : 0.2
            },
                        
            'hidden_layers' : [
                {
                    'config' : hp.choice('is_on_1', [
                        {'is_on' : False }, 
                        {
                            'is_on' : True, 
                            'neurons' : hp.quniform('neurons_1', 80, 160, 10),
                            'activation' : hp.choice('activation_1', ['relu', 'tanh', 'swish']),
                            'dropout' : 0.4
                        }
                    ]),
                }#,
                #{
                #    'config' : hp.choice('is_on_2', [
                #        {'is_on' : False }, 
                #        {
                #            'is_on' : True, 
                #            'neurons' : hp.quniform('neurons_2', 80, 160, 10),
                #            'activation' : hp.choice('activation_2', ['relu', 'tanh', 'swish']),
                #            'dropout' : 0.4
                #        }
                #    ]),
                #}
            ],
                         
            'last_layer' : {
                'activation' : 'sigmoid'
            },

            'optimizer': hp.choice('optimizer',['adadelta','adam','rmsprop']),
            'learning_rate' : 1e-3,
            'alpha': hp.quniform('alpha', 0.0005, 0.01, 0.0005)
        }


####################PRUEBA###################333


# neural_network_params = {
            
#             'first_layer' : {
#                 'neurons' : hp.quniform('first_layer_neurons', 100, 101, 1),
#                 'activation' : 'relu',
#                 'dropout' : 0.5
#             },
                        
#             'hidden_layers' : [
#                 {
#                     'config' : hp.choice('is_on_1', [
#                         {'is_on' : False }, 
#                         {
#                             'is_on' : True, 
#                             'neurons' : hp.quniform('neurons_1', 200, 205, 5),
#                             'activation' : 'relu',
#                             'dropout' : 0.5
#                         }
#                     ]),
#                 }
#             ],
                         
#             'last_layer' : {
#                 #my_relu es una funcion de activacion definida en Modulo.py que tiene como cota superior 1
#                 'activation' : 'sigmoid'
#             },

#             'optimizer': 'adam',
#             'learning_rate' : 5e-4,
#             'alpha': 1e-3
#         }







##############################################


#Prueba con una red con el top 5 features
#neural_network_params = {
#            
#            'first_layer' : {
#                'neurons' : hp.uniform('first_layer_neurons', 16, 128),
#                'activation' : hp.choice('first_layer_activation', ['relu', 'tanh', 'swish']),
#                'dropout' : hp.uniform('dropout_0', 0.25, 0.5)
#            },
#                        
#            'hidden_layers' : [
#                {
#                    'config' : hp.choice('is_on_1', [
#                        {'is_on' : False }, 
#                        {
#                            'is_on' : True, 
#                            'neurons' : hp.uniform('neurons_1', 8, 256),
#                            'activation' : hp.choice('activation_1', ['relu', 'tanh', 'swish']),
#                            'dropout' : hp.uniform('dropout_1', 0.25, 0.5)
#                        }
#                    ]),
#                },
#                {
#                    'config' : hp.choice('is_on_4', [
#                        {'is_on' : False }, 
#                        {
#                            'is_on' : True, 
#                            'neurons' : hp.uniform('neurons_4', 8, 256),
#                            'activation' : hp.choice('activation_4', ['relu', 'tanh', 'swish']),
#                            'dropout' : hp.uniform('dropout_4', 0.25, 0.5)
#                        }
#                    ]),
#                }
#            ],
#                         
#            'last_layer' : {
#                #my_relu es una funcion de activacion definida en Modulo.py que tiene como cota superior 1
#                'activation' : hp.choice('last_layer_activation', ['my_relu', 'sigmoid'])
#            },
#
#            'optimizer': hp.choice('optimizer',['adadelta','adam','rmsprop']),
#            'learning_rate' : hp.uniform('learning_rate', 1e-4, 1e-2),
#            'alpha': hp.uniform('alpha', 1e-3, 10)
#        }

#Espacio de busqueda para hiperparametros discretos -> GridSearch

#neural_network_grid_search_params = {
#            
#            'first_layer' : {
#                'neurons' : hp.quniform('first_layer_neurons', 40, 130, 10),
#                'activation' : hp.choice('first_layer_activation', ['relu', 'tanh', 'swish']),
#                'dropout' : hp.quniform('dropout_0', 0.3, 0.4, 0.1)
#            },
#                        
#            'hidden_layers' : [
#                {
#                    'config' : hp.choice('is_on_1', [
#                        {'is_on' : False }, 
#                        {
#                            'is_on' : True, 
#                            'neurons' : hp.uniform('neurons_1', 40, 160, 20),
#                            'activation' : hp.choice('activation_1', ['relu', 'tanh', 'swish']),
#                            'dropout' : hp.uniform('dropout_1', 0.3, 0.4, 0.1)
#                        }
#                    ]),
#                },
#                {
#                    'config' : hp.choice('is_on_4', [
#                        {'is_on' : False }, 
#                        {
#                            'is_on' : True, 
#                            'neurons' : hp.uniform('neurons_4', 40, 160, 20),
#                            'activation' : hp.choice('activation_4', ['relu', 'tanh', 'swish']),
#                            'dropout' : hp.uniform('dropout_4', 0.3, 0.4, 0.1)
#                        }
#                    ]),
#                }
#            ],
#                         
#            'last_layer' : {
#                #my_relu es una funcion de activacion definida en Modulo.py que tiene como cota superior 1
#                'activation' : hp.choice('last_layer_activation', ['sigmoid'])
#            },
#
#            'optimizer': hp.choice('optimizer',['adadelta','adam','rmsprop']),
#            'learning_rate' : hp.quniform('learning_rate', 0.0015),
#            'alpha': hp.quniform('alpha', 0.01, 0.1, 0.01)
#        }


neural_callbacks = [
                    
    # keras.callbacks.ModelCheckpoint("hpam_aux.hdf5", 
    #                                  monitor='val_loss', 
    #                                  verbose=0,
    #                                  save_best_only=True, 
    #                                  mode='min'),
    
    keras.callbacks.EarlyStopping(monitor = 'val_loss',
                                  min_delta=0.0001,
                                  mode='min',
                                  patience=10),
    keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                      mode='min',
                                      factor=0.5,
                                      min_delta=0.0001,
                                      patience=2,
                                      cooldown=0, 
                                      min_lr=1e-24)
]

def neural_network_fit(fold, params):
    (x_train, y_train, x_test, y_test) = fold
    input_dim = len(x_train[0])
    model = md.get_neural_network_model(params, input_dim)
    
    history = model.fit(x_train, 
                        y_train, 
                        validation_data=(x_test, y_test),
                        callbacks=neural_callbacks,
                        verbose=0,
                        epochs=150,
                        batch_size=256)

    #new_model = keras.models.load_model('hpam_aux.hdf5')
    y_pred_proba = model.predict(x_test).flatten()
    score = log_loss(y_test.flatten(), y_pred_proba, eps=1e-7)

    return [score]

## XGBoost


In [5]:
#Estos parametros estan sacados de un tuto de internet

xgboost_params = {
    'max_depth' : hp.choice('max_depth', range(5, 30, 1)),
    'learning_rate' : hp.quniform('learning_rate', 0.01, 0.5, 0.01),
    'n_estimators' : hp.choice('n_estimators', range(20, 205, 5)),
    'gamma' : hp.quniform('gamma', 0, 0.50, 0.01),
    'min_child_weight' : hp.quniform('min_child_weight', 1, 10, 1),
    'subsample' : hp.quniform('subsample', 0.1, 1, 0.01),
    'colsample_bytree' : hp.quniform('colsample_bytree', 0.1, 1.0, 0.01)
}

#Estos parametros estan sacados del repo de github

#xgboost_params = {
#        "iterations" : (5, 6),
#        'learning_rate': Real(low=0.01, high=1, prior='log-uniform'),
#        "random_seed" : (1,40000),
#        "l2_leaf_reg" : Real(low=1e-9, high=1000, prior='log-uniform'),
#        'subsample': Real(low=0.01, high=1, prior='uniform'),
#        "random_strength" : Real(low=1e-9, high=1000, prior='log-uniform'),
#        'depth': (1, 5),
#        "early_stopping_rounds" : (1, 20),
#        "border_count" : (1,65535)
#}


def xgboost_fit(fold, params):
    
    (x_train, y_train, x_test, y_test) = fold
    #No se bien como usar las matrices estas...
    #train_matrix = xgb.DMatrix(x_train,y_train)
    #test_matrix = xgb.DMatrix(x_test,y_test)
    model = md.get_xgboost_model(params)
    
    model.fit(x_train, y_train,
              eval_set=[(x_train, y_train), (x_test, y_test)],
              eval_metric='logloss',
              verbose=False)
    
    evals_result = model.evals_result()
    
    
    return evals_result['validation_1']['logloss']


# Codigo principal
### Desde este punto comienza el tuneo de hiperparametros del modelo elegido

In [6]:
#Cargamos el dataframe de training

model_name = 'Neuronales'

df_train = pd.read_pickle('Archivos/' + model_name + "_entrenamiento.pkl")

#Armamos los folds
folds = fold_split_quarter(df_train)
#Aca estamos codificando, transformando fechas y normalizando, quizas no es necesario para XGBoost
folds = prepare_folds(folds)

/home/julian/anaconda3/envs/tensorflow/lib/python3.6/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/home/julian/anaconda3/envs/tensorflow/lib/python3.6/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/home/julian/anaconda3/envs/tensorflow/lib/python3.6/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/home/julian/anaconda3/envs/tensorflow/lib/python3.6/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif

In [7]:
cols = int(folds[0][0].size / len(folds[0][0]))
print(f"El dataframe tiene {cols} columnas")

El dataframe tiene 85 columnas


In [8]:
#Configuracion de funciones para cada modelo

models = {
    'Neuronales' : {
        'model_fit_function' : neural_network_fit,
        'model_hparams' : neural_network_params
    },
    'XGBoost' : {
        'model_fit_function' : xgboost_fit,
        'model_hparams' : xgboost_params
    }
}


#Parametros generales

#epochs = 20               #Numero de iteraciones de entrenamiento para cada modelo
last_k_avg = 3             #Ultimos k valores de val_loss de cada entrenamiento a promediar
N = 5                      #Numero de iteraciones del algoritmo de tuneo.
best = None                #Donde se almacena informacion del mejor resultado del tuneo
save_best_hparams = True   #Sobrescribe el archivo de 'best_hyperparams_<model_name>.json' con el mejor de Trial
error_found = False

trials, total_iters = load_trials(N, model_name)

best_loss[0] = np.inf
for a in trials.results:
  if a['loss'] < best_loss[0]:
    best_loss[0] = a['loss']


try:

    callback = lambda params: test_model(params, 
                                         models[model_name]['model_fit_function'], 
                                         folds,
                                         models[model_name]['model_hparams'],
                                         last_k_average=last_k_avg)
    best = fmin(callback, 
                models[model_name]['model_hparams'], 
                algo=tpe.suggest, 
                max_evals=total_iters, 
                trials=trials)
except Exception as e:
    #print(f"ERROR: Modelo {model_name} no reconocido")
    print("ERROR: {}".format(str(e)))
    #No se como finalizar la ejecucion de la celda
    error_found = True

#Guardamos la informacion del tuneo para poder continuar en otro momento
if (not error_found):
    save_trials(trials, model_name)

best_params = space_eval(models[model_name]['model_hparams'], best)

#Guardamos, si corresponde, los mejores hiperparametros obtenidos hasta el momento
if (save_best_hparams and not error_found):
    ut.hyperparams_to_json(best_params, 'Archivos/' + model_name)

print("#"*50)
print("\nEl mejor modelo hasta el momento contiene los siguientes parametros:\n")
best_params

--------------------------------------------------   
Comienzo de iteracion con nuevos parametros          
{'alpha': 0.002, 'first_layer': {'activation': 'relu', 'dropout': 0.2, 'neurons': 55.0}, 'hidden_layers': ({'config': {'is_on': False}},), 'last_layer': {'activation': 'sigmoid'}, 'learning_rate': 0.001, 'optimizer': 'adadelta'}
Resultado parcial: val_loss = 0.9493907559177533     
Resultado parcial: val_loss = 0.7904856094539585     
Resultado parcial: val_loss = 0.7955843875448775     
Resultado parcial: val_loss = 0.8345521121989636     
Resultado parcial: val_loss = 0.7745430511863697     
Resultado parcial: val_loss = 0.613106583083372      
Resultado parcial: val_loss = 0.6282036206115772     
                                                     
	Resultado final (media ponderada): val_loss = 0.706874591057316

--------------------------------------------------                             
Comienzo de iteracion con nuevos parametros                                    
{'alp

{'alpha': 0.007,
 'first_layer': {'activation': 'swish', 'dropout': 0.2, 'neurons': 95.0},
 'hidden_layers': ({'config': {'is_on': False}},),
 'last_layer': {'activation': 'sigmoid'},
 'learning_rate': 0.001,
 'optimizer': 'adam'}

## Comentarios utiles
- En caso de haber perdido los parametros del mejor modelo, se los puede recuperar a partir de la variable 'best'.
- Si se perdio el resultado de la variable 'best' sera necesario entonces ejecutar la funcion fmin de hyperopt, se lo puede hacer con N=1 para que termine rapido.
- Por default hyperopt devuelve un diccionario con los mejores hiperparametros encontrados, donde la key es el label del hiperparametro y el value es, para los hiperparametros con 'hp.choice', el indice a la posicion en el vector de hp.choice definido. Para que devuelva directamente el valor del hiperparametro sera necesario utilizar la funcion 'space_eval', que se encuentra en un ejemplo en la siguiente celda.

In [9]:
#Ejemplo de como se obtienen los mejores parametros y de como se obtiene un objecto model a partir de ellos.
#best_parameters = space_eval(neural_network_params, best)
#best_parameters

In [10]:
#Ejemplo de como obtener el modelo ya creado con los mejores hiperparametros
#Dejo comentado esto para que no rompa en caso de utilizar un modelo distinto al de redes neuronales.

#best_model = get_neural_network_model(best_parameters, (df_train.shape[1] - 1))

#A partir de aca ya se puede usar el modelo, pero tener cuidado que el mismo no se encuentra entrenado.